In [7]:
split = [[0,1],[2,3]]

n = 4 #total elements
k = 2 #elements per split

split = []
for i in range(n//k):
    split.append(range(i*k,(i+1)*k))
split.append(range(n-n%k,n))

circuit_num =0

entangles = []
j=0

if j%2==0:
    for i in range(n-1):
        entangles.append((i,i+1))
    entangles.append((0,n-1))
else:
    for i in range(n-1,0,-1):
        entangles.append((i-1,i))
    entangles.append((0,n-1))

entangles


[(0, 1), (1, 2), (2, 3), (0, 3)]

In [88]:
def theta_gate(qc,theta,i):
    assert len(theta) == 3, "theta incorrect size"
    qc.rx(theta[0],i)
    qc.ry(theta[1],i)
    qc.rz(theta[2],i)
    return 0


def split_Z_gates(qc, n, j, indices,dag):
    phi = (-1)**dag*(np.pi/2)
    B = 2 in indices
    if j%2 ==0:
        qc.cz(0,1)
        qc.p(phi ,1-B)
    else:
        qc.p(phi ,1-B)
        qc.cz(0,1)
    return qc
    
def split_Z_gates_alt(qc, n, j, dag):
    phi = (-1)**dag*(np.pi/2)
    if j%2 ==0:
        qc.cz(0,1)
        qc.p(phi ,1)
        qc.p(phi ,2)
        qc.cz(2,3)
    else:
        qc.cz(2,3)
        qc.p(phi ,2)
        qc.p(np.pi/2 ,1)
        qc.cz(0,1)
    return qc           

def W_4_split(qc,theta, n, l, indices,dag):
    # Gets 2 qubit circuit, then adds W gate
    Theta_matrix = np.reshape(theta, (n,l,3))[indices,:,:]
    for j in range(l):
        for i in range(2):
            theta_gate(qc,Theta_matrix[i,j,:],i)
        split_Z_gates(qc, n, j, indices,dag[j])        
    return qc

def W_4_split_alt(qc,theta, n, l,dag):
    # Gets 4 qubit circuit, then adds W gate
    Theta_matrix = np.reshape(theta, (n,l,3))
    for j in range(l):
        for i in range(n):
            theta_gate(qc,Theta_matrix[i,j,:],i)
        split_Z_gates_alt(qc, n, j, dag[j])        
    return qc

def make_dags(i,splits):
    dags = np.zeros(splits)
    j=0
    for e in bin(i)[2:].zfill(splits):
        dags[j] = e
        j+=1
    return dags

In [73]:
for i in range(4):
    dags = np.zeros(splits)
    j=0
    for e in bin(i)[2:].zfill(splits):
        dags[j] = e
        j+=1
    print(dags)

[0. 0.]
[0. 1.]
[1. 0.]
[1. 1.]


In [9]:
import qiskit as qk
import numpy as np
n = 4
d = 1
Theta = 2*np.pi*np.random.random(n*d*3)

indices = [0,1]
qc = qk.QuantumCircuit(2)
qc = W_4_split(qc,Theta, n, d,indices, 1)
qc.draw()

# qc1 = qk.QuantumCircuit(n)
# qc1 = W_4_split_alt(qc1,Theta, n, d, 0)
# qc1.draw()

┌───────────┐ ┌────────────┐┌────────────┐              
q_0: ┤ Rx(5.222) ├─┤ Ry(3.9709) ├┤ Rz(2.8004) ├─■────────────
     ├───────────┤┌┴────────────┤├────────────┤ │ ┌─────────┐
q_1: ┤ Rx(2.606) ├┤ Ry(0.40499) ├┤ Rz(6.1214) ├─■─┤ P(-π/2) ├
     └───────────┘└─────────────┘└────────────┘   └─────────┘

In [40]:
import qiskit as qk
import numpy as np
n = 4
d = 1
Theta = 2*np.pi*np.random.random(n*d*3)

splits_per_depth = 1
N = 2**(d*splits_per_depth+1)
qcs = []

for i in range(N):
    qcs.append(qk.QuantumCircuit(n))
    dags = make_dags(i,splits_per_depth)
    for j in range(splits_per_depth):
        W_4_split_alt(qcs[i],Theta, n, d, dags[j])

In [95]:
import qiskit as qk
import numpy as np
n = 4
d = 1
Theta = 2*np.pi*np.random.random(n*d*3)

splits_per_depth = 1
N = 2**(d*splits_per_depth+1)
qcs = []

for i in range(N):
    if i%2==0:
        indices = [0,1]
    else:
        indices = [2,3]
    qc = qk.QuantumCircuit(2)
    dags = make_dags(i//2,d*splits_per_depth)
    print(dags)
    for j in range(splits_per_depth):
        W_4_split(qc,Theta, n, d, indices,dags[j*d:(j+1)*d])
    qcs.append(qc)

[0.]
[0.]
[1.]
[1.]


In [106]:
import qiskit as qk
import numpy as np
n = 4
d = 1
Theta = 2*np.pi*np.random.random(n*d*3)

splits_per_depth = 1
N = 2**(d*splits_per_depth+1)
circuits = []

for i in range(N):
    if i%2==0:
        indices = [0,1]
    else:
        indices = [2,3]
    circuit = qk.QuantumCircuit(2,2)
    dags = make_dags(i//2,d*splits_per_depth)
    print(dags)
    for j in range(splits_per_depth):
        W_4_split(circuit,Theta, n, d, indices,dags[j*d:(j+1)*d])
    circuit.z(range(2))

#Add measurement to circuit
    circuit.measure(range(2),range(2))
    circuits.append(circuit)

[0.]
[0.]
[1.]
[1.]


In [115]:
from qiskit import  Aer, transpile
shots = 1024

circuit = circuits[3]


simulator = Aer.get_backend('aer_simulator')
job = circuit
result = simulator.run(job,shots=shots).result()

#Get counts and normalize them
counts = result.get_counts()
total_counts = sum(counts.values())
counts_normalized = {state: counts[state]/total_counts for state in counts}

print(counts_normalized['00'])


0.3056640625
